(method)=
# Method

* *Present the steps and components (briefly) to create an understanding of the proposed procedure. The details can come later. Highlight what makes it unique compared to other studies in the literature (include references).*

The system identification can then be simplified into parameter identification if a Mathematical Vessel Manoeuvring Model (VMM) is assumed to describe the ship manoeuvring system. An efficient approach to build the VMM for a ship’s manoeuvres is proposed here and presented in {numref}`overview`. 
In this procedure, an initial VMM is used to solve the reversed manoeuvring problem, i.e., predicting unknown forces from known ship manoeuvres. Then, the hydrodynamic derivatives in the VMM can be identified with regression of the force polynomials on forces predicted with inverse dynamics. The Ordinary Least Square (OLS) method is used to regress the hydrodynamic derivatives. The OLS is known to be extremely sensitive to noise and outliers inevitably associated with both experimental and full-scale test data. Thereby, the focus in the present PIT is on pre-processing data with filtering rather than the regression method itself. Both the Extended Kalman Filter (EKF) and Rauch Tung Striebel (RTS) smoother are used to perform the data-processing for building a proper VMM.

```{figure} figures/method.png
---
width: 500px
name: overview
---
Flow chart over the proposed Parameter Identification Technique (PIT)
```

![](figures/method.png)

## Overview of the proposed procedure

After choosing a proper VMM model to describe a ship’s manoeuvring performance, the coefficients in those VMMs can be estimated by the proposed PIT method composed of two basic steps as in {numref}`overview`. During the estimation process, a VMM is required as the input system model in the EKF. But the identified VMM is of course not known yet which is a ''the chicken or the egg'' situation. This is solved with initial guessing and iteration.

1) First, hydrodynamic derivatives of a pre-defined format of VMM are initially guessed. To make a fair guess, the derivatives are estimated with semi-empirical formulas for a linear VMM. The VMM is used in the EKF and RTS smoother to filter all the model tests. The VMMs are assumed to have Markov property which means that future states depend only on the current state. Then the filtered data with estimated hidden states from all the model tests can be joined into a time independent dataset passed to the motion regression. The hydrodynamic derivatives are regressed on quasi-static forces from inverse dynamics giving the identified nonlinear VMM.

2) Re-run the iteration in the previous step with EKF that use the identified VMM from the previous step to replace the guessed system model in the initial stage (with Abkowitz model, Modified Abkowitz model etc.). There should be a higher belief in this model than the guessed model, so the covariance matrixes should be updated.

### Iterative Extended Kalman Filter

An example with simulation results from the steps in the iterative EKF is shown in {numref}`iterations`.

```{glue:figure} iterations
:figwidth: 1000px
:name: "iterations"

Simulation with: initial model, first and second iteration of the PIT

```

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters
from jb_helpers import df_to_myst, parameter_to_latex

In [ ]:
#id = '22773'
#ship = 'wpcc'
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'

data_raw = catalog.load(f"{ship}.{id}.data")
data_lowpass = catalog.load(f"{ship}.{id}.data_lowpass")

data = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
ek = catalog.load(f"{ship}.vmm_linear.ek")
model = catalog.load(f"{ship}.initial.vmm_linear.joined.model")

vmm_name = 'vmm_abkowitz'

In [ ]:
ships = ['wpcc','kvlcc2_hsva']

ship_datas = {}
initial_parameters = {}

for ship in ships:
       
    ship_datas[ship] = catalog.load(f"{ship}.ship_data")
    initial_parameters[ship] = catalog.load(f"{ship}.initial_parameters")
    

ship_datas = pd.DataFrame(ship_datas)
ship_datas.dropna(how='any', inplace=True)

initial_parameters = pd.DataFrame(initial_parameters)
initial_parameters.drop(index='Xthrust', inplace=True)
mask = (initial_parameters!=0).all(axis=1)
initial_parameters = initial_parameters.loc[mask].copy()

In [ ]:
data['U'] = np.sqrt(data['u']**2 + data['v']**2)
df_sim = ek.simulate(data=data, input_columns=['delta','U'], solver='Radau')

dataframes = {
    'initial guessed model': df_sim,
    'PIT iteration 1': catalog.load(f"{ship}.initial.{vmm_name}.joined.{id}.data_resimulate"),
    'PIT iteration 2': catalog.load(f"{ship}.updated.{vmm_name}.joined.{id}.data_resimulate"),
    'model test' : data,
}

keys = ['u','v','r']
error_keys = [f'$\\epsilon({key})$' for key in keys]  
for key, df_ in dataframes.items():
    df_[error_keys]=df_[keys] - data[keys]

fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_datas[ship]['L'], beam=ship_datas[ship]['B'], time_window=[0,70], N=2, ax=ax);
glue("iterations",fig, display=False)

fig = plot(dataframes, keys=error_keys, ncols=1);
plt.tight_layout()
glue("iteration_errors",fig, display=False)

rmse = {}
rmse_keys = [f'$RMSe({key})$' for key in keys] 
for key, df_ in dataframes.items():
    rmse[key] = np.sqrt((df_[error_keys]**2).mean())
    

rmse = pd.DataFrame(rmse)

r = rmse.loc['$\\epsilon(r)$']
u = rmse.loc['$\\epsilon(u)$']
v = rmse.loc['$\\epsilon(v)$']
rmse.loc['total'] = u+v+r*ship_datas[ship]['L']/2

In the following, the methods of inverse dynamics, regression and EKF used in the proposed PIT method, and their connections, are presented in detail.